In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import NFL_api
import time
import pickle

In [18]:
f= open('player_def_2014_1-17.pickle')
player_stat = pickle.load(f)
f.close()

g=open('team_def_2014_1-17.pickle')
team_stat = pickle.load(g)
g.close()

h=open('injuries_2014_1-17_MUNGED.pickle')
injuries_df = pickle.load(h)
h.close()

In [19]:
player_stat.reset_index(drop=True, inplace=True)

In [20]:
stat_list = []

for i in player_stat.columns:
    if (player_stat[i].dtype != object):
        stat_list.append(i)



In [21]:
for i in player_stat.index:
    name = player_stat.ix[i, 'name']
    week = player_stat.ix[i, 'week_num']
    if (len(injuries_df[(injuries_df['name_full']==name) & (injuries_df['week_num']==week+1)]) > 0):
        player_stat.loc[i,'injured?']=1
    else:
        player_stat.loc[i,'injured?']=0

In [22]:
grouped = player_stat.groupby(['name'])
f=lambda x: pd.rolling_mean(x,3)

for stat in stat_list:
    player_stat = player_stat.join(grouped[stat].transform(np.cumsum), how='outer', rsuffix='_cum')
    player_stat= player_stat.join(grouped[stat].transform(f).fillna(0), how='outer', rsuffix='_ravg')

In [23]:
player_stat=player_stat.join(pd.get_dummies(player_stat.position),how='outer')



In [25]:
player_stat.shape

(9962, 123)

In [26]:
f = open('player_def_2014_1-17_MUNGED_v2.pickle', 'wb')
pickle.dump(player_stat, f)
f.close()